# Abstract
>We present our submission for the IBM Quantum Open Science Prize, which focuses on the simulation of the XXX Heisenberg spin model on real quantum hardware. The keystone of our approach is the use of Incremental Structural Learning (ISL), a quantum algorithm for approximate circuit recompilation, to find circuits that produce approximately the same state as the desired time evolution circuit with significantly fewer gates. The trial solution is built and optimised incrementally, with the structure of each layer informed by the entanglement properties of the original circuit. Our open-source implementation, built as part of this challenge, is available at [github.com/abhishekagarwal2301/isl/](https://github.com/abhishekagarwal2301/isl/).
<br><br>Combining this method with measurement calibration, we achieve a 99.3\% fidelity on the tomography benchmark for the evolved state at $t=\pi$. Furthermore, to demonstrate the general applicability of our solution beyond this special case, we reconstruct the probability of the $|110\rangle$ state across the entire range $t=(0, \pi)$ on $ibmq\_jakarta$. Excitingly, despite the more complicated entanglement spectrum of the system at these intermediate times, our results match the form of the analytic solution, producing a probability on average within 2.4 percentage points of the true value.

# 0 - Imports

In [ ]:
%pip install quantum-isl #This is our open-source recompiler made for this challenge. More details in Section 3.

# Uncomment below if pip install quantum-isl fails.
# import sys
# sys.path.append('./isl/')
# sys.path.append('./isl/isl/')

%pip install mpl_interactions
%pip install ipympl
%pip install qiskit==0.34.0

If installation of quantum-isl fails, please uncomment the lines above, which will fall back to a local version of the repository zipped into our solution. Furthermore, this notebook was developed on Qiskit 0.34 and as such we have pinned this version to ensure our submission runs for the judges.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mpl_interactions.ipyplot as iplt
plt.rcParams.update({'font.size': 16})  # enlarge matplotlib fonts

# Import qubit states Zero (|0>) and One (|1>), and Pauli operators (X, Y, Z)
from qiskit.opflow import Zero, One, I, X, Y, Z

# Importing standard Qiskit modules
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister, IBMQ, execute, transpile, Aer
from qiskit.providers.aer import AerSimulator
from qiskit.test.mock import FakeJakarta
from qiskit.result import Result
from qiskit.ignis.mitigation.measurement import *
from qiskit.circuit.random import random_circuit

# Import state tomography modules
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.quantum_info import state_fidelity

# Import python scripts
import circuit_utils 
import mitigation_utils 
import analysis_utils 
import experiment_utils 
import misc_utils 
import file_utils
import job_utils

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

# 1 Introduction

### 1-1 Quantum Simulation
One of the leading uses for quantum computers will be to simulate quantum systems such as molecules or engineered materials. Actually executing a quantum simulation on a current quantum computer, however, can be difficult and error prone. Your objective for this Open Science Prize is to significantly improve the state-of-the-art for quantum simulation (details in Section 2).

The quantum system you will simulate is a quantum spin-1/2 model, where each qubit represents a quantum [spin-1/2](https://en.wikipedia.org/wiki/Spin-1/2) particle in a 1D chain. Quantum spin models have some amazing properties and uses. Computationally speaking, [certain optimization problems can be mapped](https://arxiv.org/pdf/1712.05771.pdf) to spin models and hence minimized on a quantum computer. Physically speaking, spin models show a variety of quantum behavior such as large entangled states, quantum phases of matter, quantum many-body affects, and many other open questions in physics research.

There are many different spin models, but we will focus on the [$XXX$ Heisenberg spin model](https://en.wikipedia.org/wiki/Quantum_Heisenberg_model#XXX_model). Though it is not as well known as the [transverse-field Ising model](https://en.wikipedia.org/wiki/Transverse-field_Ising_model), the $XXX$ Heisenberg model is of practical and theoretical interest. It can also be difficult to simulate, which is where you come in.

The notebook is organized as follows:
1) Section 1 (labeled 1-X) provides you with brief introductions to the $XXX$ Heisenberg model, time evolving a Hamiltonian, and decomposing a time evolution operator into gates for a quantum computer to execute.
2) Section 2 (labeled 2-X) outlines the specific requirements and boundaries associated with completing and competing in the Open Science Prize. Continuing from Section 1, the notebook showns an example solution.

### 1-2 The $XXX$ Heisenberg Spin Model

To simulate a quantum system, we first need to define how the quantum particles that make up the system interact with each other. In quantum physics, this is done by defining a [Hamiltonian operator](https://en.wikipedia.org/wiki/Hamiltonian_(quantum_mechanics)). For this Open Science Prize, you will work with the $XXX$ Heisenberg Hamiltonian $H_{\text{Heis}}$ as defined below

$$
\begin{equation}
H_{\text{Heis}} = \sum_{\langle ij \rangle}^{N} J \left(\sigma_x^{(i)}\sigma_x^{(j)} + \sigma_y^{(i)}\sigma_y^{(j)} + \sigma_z^{(i)}\sigma_z^{(j)}\right).
\end{equation}
$$

$N$ is the number of spin-1/2 particles in model. The operators $\sigma_x$, $\sigma_y$, and $\sigma_z$ are [Pauli operators](https://en.wikipedia.org/wiki/Pauli_matrices) where the $i$ and $j$ superscripts label which qubit they act on. For example, $\sigma_x^{(1)}$ would be the $\sigma_x$ operator acting on only qubit 1 (which is the 2nd qubit since indexing starts at 0). This version of the general Heisenberg spin model is called $XXX$ because the same $J$ value multiplies each pair of Pauli operators. The sum notation $\langle ij \rangle$ means the sum is over nearest neighbors (only qubits next to each other interact), and $J$ is the interaction strength, which we will set $J=1$.

You will work with the explicit case of $N=3$ with the 3 spins arranged in a line. Written out fully, the Hamiltonian is
$$
H_{\text{Heis3}} = \sigma_x^{(0)}\sigma_x^{(1)} + \sigma_x^{(1)}\sigma_x^{(2)} + \sigma_y^{(0)}\sigma_y^{(1)} + \sigma_y^{(1)}\sigma_y^{(2)} + \sigma_z^{(0)}\sigma_z^{(1)} + \sigma_z^{(1)}\sigma_z^{(2)}.
$$
Now that we have a Hamiltonian ($H_{\text{Heis3}}$), we can use it to determine how the quantum system of 3 spin-1/2 particles changes in time.

*See the supplementary material for further details on $H_{\text{Heis}}$, $H_{\text{Heis3}}$ and its matrix repsentation.*

In [ ]:
# Returns the matrix representation of the XXX Heisenberg model for 3 spin-1/2 particles in a line
def H_heis3():
    # Interactions (I is the identity matrix; X, Y, and Z are Pauli matricies; ^ is a tensor product)
    XXs = (I^X^X) + (X^X^I)
    YYs = (I^Y^Y) + (Y^Y^I)
    ZZs = (I^Z^Z) + (Z^Z^I)
    
    # Sum interactions
    H = XXs + YYs + ZZs
    
    # Return Hamiltonian
    return H

### 1-3 Time Evolution

Knowing the Hamiltonian, we can determine how quantum states of that system evolve in time by solving the [Schrödinger equation](https://en.wikipedia.org/wiki/Schrödinger_equation)
$$
i\hbar \dfrac{d}{dt}|\psi(t)\rangle = H |\psi(t)\rangle
$$

For simplicity, let's set $\hbar = 1$. We know that the Hamiltonian $H_{\text{heis3}}$ does not change in time, so the solution to the Schrödinger equation is an exponential of the Hamiltonian operator
$$
\begin{align}
U_{\text{Heis3}}(t) &= e^{-it H_\text{Heis3}} = \exp\left(-it H_\text{Heis3}\right) \\
U_{\text{Heis3}}(t) &= \exp\left[-it \sum_{\langle ij \rangle}^{N=3} \left(\sigma_x^{(i)}\sigma_x^{(j)} + \sigma_y^{(i)}\sigma_y^{(j)} + \sigma_z^{(i)}\sigma_z^{(j)}\right) \right] \\
U_{\text{Heis3}}(t) &= \exp\left[-it \left(\sigma_x^{(0)}\sigma_x^{(1)} + \sigma_x^{(1)}\sigma_x^{(2)} + \sigma_y^{(0)}\sigma_y^{(1)} + \sigma_y^{(1)}\sigma_y^{(2)} + \sigma_z^{(0)}\sigma_z^{(1)} + \sigma_z^{(1)}\sigma_z^{(2)}\right) \right]
\end{align}
$$.

Now that we have the time evolution operator $U_{\text{Heis3}}(t)$, we can simulate changes in a state of the system ($|\psi(t)\rangle$) over time $|\psi(t)\rangle = U_{\text{Heis3}}(t)|\psi(t=0)\rangle$. 

*See the supplementary material for more information on exponentiating an operator or matrix.*

In [ ]:
# Returns the matrix representation of U_heis3(t) for a given time t assuming an XXX Heisenberg Hamiltonian for 3 spins-1/2 particles in a line
def U_heis3(t):
    # Compute XXX Hamiltonian for 3 spins in a line
    H = H_heis3()
    
    # Return the exponential of -i multipled by time t multipled by the 3 spin XXX Heisenberg Hamilonian 
    return (t * H).exp_i()

### 1-4 Classical Simulation of $H_{\text{Heis3}}$

Now that we have the operator $U_{\text{Heis3}}(t)$ which describes how any state of the quantum system changes in time, we can simulate the $XXX$ Heisenberg spin model! Just for fun, let's simulate the system using a classical computer first.

(Fun fact: Notice the size of $U_{\text{Heis3}}(t)$ in matrix form. It's representated by an $8$x$8$ matrix. This is because there are $2^3=8$ states in the $N=3$ system. If the simulation were of $50$ particles ($N=50$), $U_{\text{Heis}}(t)$ would be approximately $10^{15}$x$10^{15}$! Well beyond the capacity of today's computers.)

In [ ]:
%matplotlib ipympl

# Define array of time points
ts = np.linspace(0, np.pi, 100)

# Define initial state |110>
initial_state = One^One^Zero

# Compute probability of remaining in |110> state over the array of time points
 # ~initial_state gives the bra of the initial state (<110|)
 # @ is short hand for matrix multiplication
 # U_heis3(t) is the unitary time evolution at time t
 # t needs to be wrapped with float(t) to avoid a bug
 # (...).eval() returns the inner product <110|U_heis3(t)|110>
 #  np.abs(...)**2 is the modulus squared of the innner product which is the expectation value, or probability, of remaining in |110>
probs_110 = [np.abs((~initial_state @ U_heis3(float(t)) @ initial_state).eval())**2 for t in ts]

# Plot evolution of |110>
plt.plot(ts, probs_110)
plt.xlabel('time')
plt.ylabel(r'probability of state $|110\rangle$')
plt.title(r'Evolution of state $|110\rangle$ under $H_{Heis3}$')
plt.grid()
plt.show()

### 1-5 Decomposition of $U_{\text{Heis3}}(t)$ Into Quantum Gates

To execute $U_{\text{Heis3}}(t)$ on a [circuit model](https://qiskit.org/documentation/apidoc/circuit.html) quantum computer, we must decompose $U_{\text{Heis3}}(t)$ into a product of single and two-qubit gates that are native to that quantum computer. There are a [variety of techniques](https://en.wikipedia.org/wiki/Hamiltonian_simulation) to do this. For this Open Science Prize, you will only use [Trotterization](https://en.wikipedia.org/wiki/Hamiltonian_simulation#Product_Formulas) (which is sometimes refered to as Suzuki-Trotterization decomposition, Trotter formulas, product formulas, and other names).

Below, we show an example Trotterization as outlined in \[1-2\].

Since the [Pauli operators do not commute](https://en.wikipedia.org/wiki/Pauli_matrices#Commutation_relations) with each other, the exponential $U_{\text{Heis3}}(t)$ cannot be split into a product of simpler exponentials. However, we can approximate $U_{\text{Heis3}}(t)$ as a product of simpler exponentials through Trotterization. Consider a subsystem of 2 spin-1/2 particles within the larger 3 spin system. The Hamiltonian on spins $i$ and $j$ ($i,j \in \{0,1,2\}$) would be $H^{(i,j)}_{\text{Heis2}} = \sigma_x^{(i)}\sigma_x^{(j)} + \sigma_y^{(i)}\sigma_y^{(j)} + \sigma_z^{(i)}\sigma_z^{(j)}$. Rewritting $U_{\text{Heis3}}(t)$ in terms of the two possible subsystems within the total $N=3$ system you will simulate,

$$
U_{\text{Heis3}}(t) = \exp\left[-i t \left(H^{(0,1)}_{\text{Heis2}} + H^{(1,2)}_{\text{Heis2}} \right)\right].
$$

$H^{(0,1)}_{\text{Heis2}}$ and $H^{(1,2)}_{\text{Heis2}}$ do not commute, so $U_{\text{Heis3}}(t) \neq \exp\left(-i t H^{(0,1)}_{\text{Heis2}}\right) \exp\left(-i t H^{(1,2)}_{\text{Heis2}} \right)$. But, this product decomposition can be approximated with Trotterization which says $U_{\text{Heis3}}(t)$ is approximately a short evolution of $H^{(0,1)}_{\text{Heis2}}$ (time = $t/n$) and followed by a short evolution of $H^{(1,2)}_{\text{Heis2}}$ (time = $t/n$) repeated $n$ times

$$
\begin{align}
U_{\text{Heis3}}(t) &= \exp\left[-i t \left(H^{(0,1)}_{\text{Heis2}} + H^{(1,2)}_{\text{Heis2}} \right)\right] \\
U_{\text{Heis3}}(t) &\approx \left[\exp\left(\dfrac{-it}{n}H^{(0,1)}_{\text{Heis2}}\right) \exp\left(\dfrac{-it}{n}H^{(1,2)}_{\text{Heis2}} \right)\right]^n.
\end{align}
$$

$n$ is the number of Trotter steps, and as $n$ increases, the approximation becomes more accurate. (Note that how a untiary is split up into subsystems for Trotterization is not necessarily unique.) The decomposition goes further. Within each 2 spin subsystems, the Pauli operator pairs ($\sigma_x^{(i)}\sigma_x^{(j)}$, $\sigma_y^{(i)}\sigma_y^{(j)}$, and $\sigma_z^{(i)}\sigma_z^{(j)}$) commute. This means we can decompose the exponential of a subsystem Hamiltonian ($H^{(i,j)}_{\text{Heis2}}$) into a product of even simpler exponentials getting us closer to a gate implementation of $U_{\text{Heis3}}(t)$

$$
\begin{align}
U_{\text{Heis3}}(t) &\approx \left[\exp\left(\dfrac{-it}{n}H^{(0,1)}_{\text{Heis2}}\right) \exp\left(\dfrac{-it}{n}H^{(1,2)}_{\text{Heis2}} \right)\right]^n \\
U_{\text{Heis3}}(t) &\approx \left[\exp\left(\dfrac{-it}{n}\left(\sigma_x^{(0)}\sigma_x^{(1)} + \sigma_y^{(0)}\sigma_y^{(1)} + \sigma_z^{(0)}\sigma_z^{(1)}\right)\right) \exp\left(\dfrac{-it}{n}\left(\sigma_x^{(1)}\sigma_x^{(2)} + \sigma_y^{(1)}\sigma_y^{(2)} + \sigma_z^{(1)}\sigma_z^{(2)}\right)\right)\right]^{n} \\
U_{\text{Heis3}}(t) &\approx \left[\exp\left(\dfrac{-it}{n}\sigma_x^{(0)}\sigma_x^{(1)}\right) \exp\left(\dfrac{-it}{n}\sigma_y^{(0)}\sigma_y^{(1)}\right) \exp\left(\dfrac{-it}{n}\sigma_z^{(0)}\sigma_z^{(1)}\right) \exp\left(\dfrac{-it}{n}\sigma_x^{(1)}\sigma_x^{(2)}\right) \exp\left(\dfrac{-it}{n}\sigma_y^{(1)}\sigma_y^{(2)}\right) \exp\left(\dfrac{-it}{n}\sigma_z^{(1)}\sigma_z^{(2)}\right)\right]^{n}
\end{align}
$$

For simplicity, and to use more common notation, let's rename the products with $XX(2t) = \exp\left(-it \sigma_x\sigma_x\right)$, $YY(2t) = \exp\left(-it \sigma_y\sigma_y\right)$, and $ZZ(2t) = \exp\left(-it \sigma_z\sigma_z\right)$ and rewrite the Trotterized $U_{\text{Heis3}}(t)$
$$
U_{\text{Heis3}}(t) \approx \left[XX\left(\frac{2t}{n}\right)^{(0,1)} YY\left(\frac{2t}{n}\right)^{(0,1)} ZZ\left(\frac{2t}{n}\right)^{(0,1)} XX\left(\frac{2t}{n}\right)^{(1,2)} YY\left(\frac{2t}{n}\right)^{(1,2)} ZZ\left(\frac{2t}{n}\right)^{(1,2)} \right]^{n}
$$
And that's it! We have now approximately decomposed $U_{\text{Heis3}}(t)$ into two-qubit gates $XX(t)$, $YY(t)$, and $ZZ(t)$. These gates are not native to superconducting qubits, but in Section 2, they will be further decomposed into native single and two-qubit gates with *further details found in the supplementary material.*


\[1\] Y. Salathe, et al., *Digital Quantum Simulation of Spin Models with Circuit Quantum Electrodynamics*, [Phys. Rev. X **5**, 021027 (2015)](https://link.aps.org/doi/10.1103/PhysRevX.5.021027)

\[2\] F. Tacchino, et al., *Quantum Computers as Universal Quantum Simulators: State-of-the-Art and Perspectives*, [Adv. Quantum Technol. *3* 3 (2020)](https://doi.org/10.1002/qute.201900052) \[[free arxiv version](https://arxiv.org/abs/1907.03505)\]


# 2 The Open Science Prize
### 2-1 Contest Details
The Open Science Prize requirements and judging criteria can be found in the terms and conditions when registering on the [website](https://ibmquantumawards.bemyapp.com/#/event). Below is an outline of those requirements.

You are to time evolve the state $|110\rangle$ to time $t=\pi$ under the $XXX$ Heisenberg model Hamiltonian $H_{\text{heis3}}$ as defined in Section 1. This will be done on the ibmq_jakarta quantum computer. From left to right, the qubits in $|110\rangle$ correspond to ibmq_jakarta's qubits 5, 3, and 1. The evolution must be done using Trotterization with at least 4 steps ($n\geq4$). You will be judged on the clarity, originiality, and creativity of your solution, as well as the fidelity of the evolved state at time $t=\pi$ using [state tomography](https://qiskit.org/documentation/tutorials/noise/8_tomography.html). State tomography is a way of determining the exact quantum state--even phase information--at the end of a quantum circuit by running the circuit multiple times and measuring in different bases. The closer your final quantum state to the expected state, the higher the fidelity. Ibmq_jakarta's qubits 0, 2, 4, and 6 will not be evaluated by the state tomography calculation, so you may use them as you wish. You do not need to follow the same decomposition given in the example above, but your solution must be a Trotterization of $U_{\text{heis3}}(t)$ with at least 4 steps ($n\geq4$). You may find it useful to implement techniques that reduce errors and improve qubit coherence.

### 2-3 Quantum Devices
Connect to IBM provider and connect to a real or simulated backend. Final submissions must be run on a real backend, but simulated devices are faster for debugging and testing.

In [ ]:
statevector = Aer.get_backend('statevector_simulator')
fake_backend = AerSimulator.from_backend(FakeJakarta())


## 3 Our Solution

Our solution uses approximate quantum circuit recompilation (AQCR) and measurement calibration. This minimalistic approach is informed by the observation that at the current stage in quantum hardware maturity, techniques which require increased circuit depth for more accurate simulation or stronger error mitigation, are unlikely to yield benefits that outweight the effects of increasing noise with each additional gate. Conversely, our error mitigation approach can be summarised almost entirely by trying to produce the same time evolution with as few gates as possible. This can lead to significant reductions in the noise accumulated on real devices, which grows exponentially with the number of logic gates.

Our solution uses an AQCR algorithm known as Incremental Structural Learning (ISL). More details about ISL as well as our submission can be found in the supplementary PDF. The aim of ISL is to find a quantum circuit which approximates the action of a target circuit on a given input to within some tolerance. Typically, this problem is approached by trialing different parameterised circuits, known as ansatzes, however this can prove expensive and runs into trainability issues. Instead of optimising a fixed circuit, ISL incrementally builds and optimises an ansatz informed by the entanglement present in the system. Here, each layer of the ansatz is a thinly dressed CNOT gate - a CNOT gate surrounded by 4 single qubit rotations. This process repeats layer-by-layer until the desired threshold is met. This threshold is based on the overlap between the state produced by the original circuit and the ISL solution. More formally, the cost we are minimising is

$$
    C = 1 - |\langle 0 | V^{\dagger} U | 0 \rangle |^2
$$

where $V$ and $U$ are the recompiled and original circuit respectively. ISL terminates once $C$ is less than or equal to a value we call the "sufficient cost".

Whilst the ideas behind the ISL method have been available publicly since 2020 and have been applied to research problems before, their implementation has previously remained in the private domain. Therefore, we consider a significant part of our submission to be the development of an open-source and easy to use implementation of ISL, which we hope others can use and extend further. Rather than import supplementary Python files, in the spirit of the Open Science Prize, we have made the core of our solution available through the public repository https://github.com/abhishekagarwal2301/isl and have also published it on PyPi. 

## 3-0 Introduction to our ISL implementation

Below we provide a small example to provide intuition for how this method works, as well as a walkthrough of how to use our built package. Here we are going to recompile a random circuit.

In [ ]:
# create random circuit, then transpile into fixed gate set.
qc_isl_ex = random_circuit(2,10, seed=1234)

# run on statevector backend then obtain statevector from result
state = execute(qc_isl_ex, statevector).result().get_statevector()

We've also saved the state produced by this random circuit, so we can explicitly show later how well our recompiled solution reproduces the state or not later. Let's print some information about our random circuit.

In [ ]:
print('Original circuit')
print(qc_isl_ex)
print("Ops of original circuit: {}".format(qc_isl_ex.count_ops()))
print("Circuit depth of original circuit: {}".format(qc_isl_ex.depth()))

Next we will import and build our ISL recompiler.

In [ ]:
from isl.recompilers import ISLConfig, ISLRecompiler
import logging

logging.getLogger("isl").setLevel(logging.DEBUG)

# Build our configuration
config = ISLConfig(sufficient_cost=1e-1)

# Build the recompiler and do the recompilation
results = ISLRecompiler(qc_isl_ex, isl_config=config, backend=statevector).recompile()

# Extract the results
recompiled_qc_isl_ex = results['circuit']
print('\n Recompiled circuit with sufficient cost of 0.1')
print(recompiled_qc_isl_ex)
print("Ops of ISL 90% overlap circuit: {}".format(recompiled_qc_isl_ex.count_ops()))
print("Circuit depth of ISL 90% overlap circuit: {}".format(recompiled_qc_isl_ex.depth()))

Looking at the above block, we first built an `ISLConfig` object and defined a `sufficient_cost=0.1`. This means that if the cost function (as defined previously) goes below 0.1, we consider the recompilation completed. Or put another way, if the current trial circuit produces a statevector with 90% overlap to the original circuit output or better, we are finished.

`ISLConfig` can also take a number of other useful options, some of which are described below:

`max_layers`: Maximum number of layers where each layer has a
thinly dressed cnot gate

`max_2q_gates`: ISL will terminate if the number of 2 qubit
gates reaches this value

`cost_improvement_tol`: The minimum relative cost improvement to continue adding layers, designed to abort if stuck in a local minima.

After this we then built an `ISLRecompiler`, passing in the circuit to recompile, the configuration as well as the backend we want to use. Note that the recompilation algorithm can itself be run on measurement based devices, including real hardware, through the use of SWAP tests for measuring the overlap and circuit based entanglement measures. However, because of device noise, the most accurate results for this competition will be obtained if we run ISL on the Statevector simulator.

`ISLRecompiler` has some other useful arguments, inclduing:

`entanglement_measure`: The entanglement measurement method to use for quantifying local entanglement, chosen from entanglement of formation, concurrency or negativity.

`coupling_map`: A user supplied coupling map which will restrict which qubits the recompiled solution can act on. Excellent for avoiding SWAP gates, although not required for the final benchmark.

`custom_layer_2q_gate`: Enables specifying a different two-qubit "layer" instead of the thinly-dressed CNOT gate to incrementally build up the solutionwith.

Finally, we obtained the result and extracted the circuit from it. The result is actually a dictionary containing other intersting information too.

In [ ]:
import pprint

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(results)

We can verify that the fidelity of the states produced by the two circuits is greater than 90%

In [ ]:
approx_state = execute(recompiled_qc_isl_ex, statevector).result().get_statevector()
print('\n Fidelity after recompilation with sufficient cost of 0.1: {}'.format(state_fidelity(state, approx_state)))

So we have found a (roughly) approximate equivalent to the random circuit using only 1 CNOT gate! Next we repeat recompilation of the circuit, this time with the more strict condition of `sufficient_cost=0.01`.

In [ ]:
config_2 = ISLConfig(sufficient_cost=1e-2)
results_2 = ISLRecompiler(qc_isl_ex, isl_config=config_2, backend=statevector).recompile()
recompiled_qc_isl_ex_2 = results_2['circuit']

slightly_less_approx_state = execute(recompiled_qc_isl_ex_2, statevector).result().get_statevector()

print('\n Recompiled circuit with sufficient cost of 0.01')
print(recompiled_qc_isl_ex_2)
print("Ops of ISL 99% overlap circuit: {}".format(recompiled_qc_isl_ex_2.count_ops()))
print("Circuit depth of ISL 99% overlap circuit: {}".format(recompiled_qc_isl_ex_2.depth()))

# compute the fidelity of the approximate statevectors with the exact statevector from the original circuit.
print('Fidelity after recompilation with sufficient cost of 0.01: {}'.format(state_fidelity(state, slightly_less_approx_state)))

print('\n The complete ISL result object for a sufficient cost of 0.01:')
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(results_2)

Here an approximate equivalent to the random circuit has been found, this time with much higher accuracy. Despite the minimum cost being 0.01 (99% overlap), the solution found on the final layer happened to be significantly higher. We notice however, in order to achieve this high overlap an extra layer was needed. In the case of > 2 qubits, this could result in significantly more layers and subsequenetly more gates to run.

### 3-1 Determining Minimum Number of Necessary Trotter Steps 

Now that we understand how ISL can provide shallower approximate equivalents of quantum circuits, we move on to the problem at hand. If we are to simulate the evolution of the XXX model on real quantum hardware there will be two sources of error: from the device and from our Suzuki-Trotter decomposition. In this section, we'd first like to eliminate the latter, so that we can focus purely on error mitigation of the device. Since the Trotter error grows as the size of the discrestised time slice $\delta t$, we aim to find the number of time slices required for it to be negligible. Here we define negligible Trotter error as the minimum number of Trotter steps required to reproduce the original |110> evolution graph within visual convergence. This is an alternative to other methods of estimating Trotter error.

#### **NB**: The results in section 3 are all loaded from files containing the results of experiments done over the course of the contest. This is to save you a rather lengthy runtime. Our two main results, both run on the real device, can be reproduced in section 4. Every other result in section 3, run on the statevector and Fake Jakarta simulators, can be reproduced in the 'ibmq-qsim-challenge-run-supporting' notebook. 

In [ ]:
''' 
    Looking at the problem in more generality, what is the minimum Trotter step required to reproduce whole time 
    evolution of probability within some margin of error
    
    
    Produce convergence of the default Trotter solution (or any Trotterisation, make it plug & play) to the exact
    probability evolution. This is a bit redundant for the code but is minimum necessary for us to justify the 
    generality of ISL
''' 
target_time = np.pi
init_trotter_steps = 4
max_trotter_steps = 36
num_qubits = 7
backend = statevector
shots = 8192
reps = 8

yss_min_ts = []
xss_min_ts = []
markers_min_ts = []
labels_min_ts = []
alphas_min_ts = []

for trot_step in range(init_trotter_steps, max_trotter_steps):
    fnames_min_ts = [file_utils.get_partial_trotter_circuit_state_probs_filename('0','pi', trot_step, i, backend.name(), shots, reps) for i in range(trot_step + 1)]
    state_probs_at_each_trotter_step = []
    
    print('\nLoading files containing partial trotter circuit results with a max_trotter_step of {}'.format(trot_step))
    [print('    ' + f) for f in fnames_min_ts]

    try:
        state_probs_at_each_trotter_step = [np.load(f + '.npy', allow_pickle=True).tolist() for f in fnames_min_ts]
    except FileNotFoundError: # if existing results not found, do simulations from scratch
        print('File not found, doing new simulations')
        # returns jobs for each incrementally growing Trotter circuit 
        # e.g. -T(pi/20)-, -T(pi/20)-T(pi/20),....
        # Each of these approximates the evolution of the giving Hamiltonian at increasing time steps
        # Having a larger 'max_trotter_steps' will mean more of the circuits, each parameterised by a smaller time slice.
        jobs_for_each_trotter_step = experiment_utils.simulate_heisenberg_xxx_over_time(target_time, trot_step, num_qubits, backend, shots, reps, False)
        
        for f_jobs in zip(fnames, jobs_for_each_trotter_step): 
            # get state probabilities from counts and average over multiple reps
            state_probs = analysis_utils.state_probabilities_from_circuit_multi_reps(f_jobs[1])
            np.save(f_jobs[0], state_probs)
            state_probs_at_each_trotter_step.append(state_probs)
                    
    target_state = '0101000'
    yss_min_ts.append([state_probs[target_state] if target_state in state_probs else 0 for state_probs in state_probs_at_each_trotter_step])
    xss_min_ts.append([i*(target_time / trot_step) for i in range(0, trot_step + 1)])
    labels_min_ts.append(trot_step)
    markers_min_ts.append('g+')
    alphas_min_ts.append(0.5)
    
yss_min_ts.append(probs_110)
xss_min_ts.append(ts)
labels_min_ts.append('Exact')
markers_min_ts.append('-')
alphas_min_ts.append(1)


In [ ]:
%matplotlib ipympl

# define the callback functions for the slider
def f_x(trotter_step):
    return xss_min_ts[trotter_step - init_trotter_steps]

def f_y(x, trotter_step):
    return yss_min_ts[trotter_step - init_trotter_steps]


xlabel = 'Time'
ylabel = 'Probability of 110'

plt.plot(xss_min_ts[-1], yss_min_ts[-1], markers_min_ts[-1], alpha=alphas_min_ts[-1], label='Exact')

all_trotter_steps = list(range(init_trotter_steps, max_trotter_steps)) # define the slider value range
controls = iplt.plot(f_x, f_y, 'ko', trotter_step=all_trotter_steps, clip_on=False, label='Trotter Approximation')

plt.legend(loc='best', prop={'size': 10})
plt.xlim(0,np.pi)
plt.xlabel(xlabel)
_ = plt.ylabel(ylabel)


<span style="color:blue">**If any of the graphs in this notebook do not show up, or the slider doesn't work, please rerun their cells individually. If you are in the same session as when you installed the libraries in the first cell, please reopen the notebook from scratch.**</span>

Adjusting the slider we can see how increasing the number of Trotter steps changes the solution in two ways. Firstly, increased Trotter steps leads to smaller time slices, producing more discrete points across the evolution. Most importantly however, the smaller time slices leads to reduced error from the decomposition, such that we observe convergence with the analytic solution as the slider is increased. Overall, we see that 35 Trotter steps is sufficient for reproducing the probability evolution from 0 to pi to a level where the error will be insignificant compared to the device noise.

In [ ]:
min_trotter_steps_visual_convergence = 35

Now we have an idea of the number of Trotter steps we need to approximate time evolution within our error threshold. We will recompile the circuit corresponding to this number of steps with ISL and see how close the result is on the noise free backend.

### 3-2 ISL Reproduces Full Probability Evolution

In [ ]:
'''
    Produce comparison between probability evolution of 110 state with time for an exact solution, the default 
    Trotter step and the default Trotter step recompiled with ISL.
'''
target_time = np.pi
trotter_steps = min_trotter_steps_visual_convergence
num_qubits = 7
backend = statevector
shots = 8192
reps = 8
sufficient_cost = 1e-3
target_state = '0101000'

# load files to generate plots
fs_full_pe = [file_utils.get_partial_trotter_circuit_state_probs_filename('0','pi', trotter_steps, i, backend.name(), shots, reps) for i in range(trotter_steps+1)]
fs_isl_full_pe = [file_utils.get_partial_trotter_circuit_state_probs_isl_filename('0','pi', trotter_steps, i, backend.name(), shots, reps, sufficient_cost) for i in range(trotter_steps+1)]

print('\nLoading files containing trotter circuit results with a trotter_step of {}'.format(trotter_steps))
[print('    ' + f) for f in fs_full_pe]
print('\nLoading files containing recompiled trotter circuit results with a trotter_step of {}'.format(trotter_steps))
[print('    ' + f) for f in fs_isl_full_pe]

try: # try and load the results using non recompiled trotter circuits
    state_probs_at_each_trotter_step = []
    for f in fs_full_pe:
        probs = np.load(f + '.npy', allow_pickle=True).tolist()
        state_probs_at_each_trotter_step.append(probs)
except FileNotFoundError: 
    print('couldnt find file')

try: # try and load results using recompiled trotter circuits
    state_probs_at_each_trotter_step_isl = [np.load(f + '.npy', allow_pickle=True).tolist() for f in fs_isl_full_pe]
except FileNotFoundError: # do simulations from scratch
    print('Running simulations isl')
    # note that the 'isl' flag is now True
    jobs_for_each_trotter_step_isl = experiment_utils.simulate_heisenberg_xxx_over_time(target_time, trotter_steps, num_qubits, backend, shots, reps, True, sufficient_cost)
    state_probs_at_each_trotter_step_isl = []

    for fs_jobs in zip(fs_isl_full_pe, jobs_for_each_trotter_step_isl):
        state_probs = analysis_utils.state_probabilities_from_circuit_multi_reps(fs_jobs[1])
        state_probs_at_each_trotter_step_isl.append(state_probs)
        np.save(fs_jobs[0], state_probs)
        
# obtain probability of the chosen target state from the complete set of state probabilities
# for the non recompiled case
ys_full_pe = [state_probs[target_state] if target_state in state_probs else 0 for state_probs in state_probs_at_each_trotter_step]
# for the recompiled case
ys_isl_full_pe = [state_probs[target_state] if target_state in state_probs else 0 for state_probs in state_probs_at_each_trotter_step_isl]
# collect the time steps
xss_full_pe = [ts] + [[i*(target_time/trotter_steps) for i in range(0, trotter_steps+1)] for _ in range(2)]
# collect the probabilities
yss_full_pe = [probs_110, ys_full_pe, ys_isl_full_pe]

labels_full_pe = ['Exact','Direct Trotter Implementation','Direct Trotter Implementation + ISL']
markers_full_pe = ['-','o', 'x']
title_full_pe = 'Max Trotter steps: {}, suff cost: {}, shots: {}. reps: {}'.format(trotter_steps, sufficient_cost, shots, reps)


In [ ]:
# reset the matplotlib backend
%matplotlib ipympl

# plot multiple datasets on same graph
misc_utils.xys_plot(plt, xss_full_pe, yss_full_pe, 'Time', 'Probability of 110',labels_full_pe, markers_full_pe, title_full_pe, alphas=[1 for i in range(len(xss_full_pe))])


Here we see that for a trial sufficient cost of 0.001 (99.9% overlap between the original and recompiled states), our recompiled solution is able to reproduce the observed population over the full evolution. Most importantly, it is able to do so with significantly shallower circuits than the direct Trotterised implementation. Here the Trotterised evolution circuits contain 10 CNOT gates per Trotter step (transpiling with optimisation level 3). By comparison, ISL finds an approximately equivalent circuit to within 99.9% overlap with on average 3 CNOT gates, for **any** number of Trotter steps. This difference is most notable when obtaining the population at the final time $t=\pi$, where the direct Trotterised implementation requires 10*35=350$ CNOT gates.

Now let's look at how both methods perform on the fake Jakarta backend.

### 3-3 Introducing noise: simulating on Fake Jakarta

In this section we rerun our solution, attempting to build the full time evolution graph on the emulated noisy Fake Jakarta.

The presence of device noise does introduce a trade off for ISL: a lower sufficient cost will lead to a recompiled circuit which outputs a state much closer to the target state, however, for more complex target states this could lead to deeper circuits. These additional gates will in turn produce noise that offsets the theoretical benefit of the higher overlap between the target and recompiled circuits. To determine the optimal trade off between the accuracy of the recompilation and depth of circuit, we repeat our experiment for increasing ISL sufficient costs. This trade off can be explored using the slider in the graph below.

In [ ]:
'''
    Here we produce the probability evolution graph for the |110> state on the emulated Fake Jakarta.
    We compare performance using Trotterised evolution circuits with and without approximate circuit recompilation. 
    
    rdp : real device performance
'''
backend = fake_backend
shots = 8192
reps = 8

fs_rdp = [file_utils.get_partial_trotter_circuit_state_probs_filename('0','pi', trotter_steps, i, backend.name(), shots, reps) for i in range(trotter_steps+1)]
print('\nLoading files containing trotter circuit results with a trotter_step of {}'.format(trotter_steps))
[print('    ' + f) for f in fs_rdp]

# try and load noisy simulation results using un recompiled trotter circuits
try:
    state_probs_at_each_trotter_step = []
    for f in fs_rdp:
        probs = np.load(f + '.npy', allow_pickle=True).tolist()
        state_probs_at_each_trotter_step.append(probs)
except FileNotFoundError: 
    print('couldnt find file for default Trotter step')

ys_rdp = [state_probs[target_state] if target_state in state_probs else 0 for state_probs in state_probs_at_each_trotter_step]


In [ ]:
ys_isl_for_each_suff_cost_rdp = []
sufficient_costs = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6]

for sufficient_cost in sufficient_costs:
    fs_isl_rdp = [file_utils.get_partial_trotter_circuit_state_probs_isl_filename('0','pi', trotter_steps, i, backend.name(), shots, reps, sufficient_cost) for i in range(trotter_steps+1)]
    
    print('\nLoading files containing recompiled trotter circuit results with a trotter_step of {} and sufficient cost of {}'.format(trotter_steps, sufficient_cost))
    [print('    ' + f) for f in fs_isl_rdp]
    
    # try and load noisy simulation results using recompiled trotter circuit
    try:
        state_probs_at_each_trotter_step_isl = [np.load(f + '.npy', allow_pickle=True).tolist() for f in fs_isl_rdp]
    except FileNotFoundError:
        print('Running simulations for recompiled Trotter circuit')
        jobs_for_each_trotter_step_isl = experiment_utils.simulate_heisenberg_xxx_over_time(target_time, trotter_steps, num_qubits, backend, shots, reps, True, sufficient_cost)
        state_probs_at_each_trotter_step_isl = []

        for fs_jobs in zip(fs_isl_rdp, jobs_for_each_trotter_step_isl):
            state_probs = analysis_utils.state_probabilities_from_circuit_multi_reps(fs_jobs[1])
            state_probs_at_each_trotter_step_isl.append(state_probs)
            np.save(fs_jobs[0], state_probs)

    ys_isl_for_each_suff_cost_rdp.append([state_probs[target_state] if target_state in state_probs else 0 for state_probs in state_probs_at_each_trotter_step_isl])


In [ ]:
# reset the matplotlib backend
%matplotlib ipympl

# define callback function for the slider
def f_y(x, sufficient_cost):
    title = 'Max Trotter steps: {}, suff cost: {}, shots: {}. reps: {}'.format(trotter_steps, sufficient_cost, shots, reps)
    plt.title(title)
    return ys_isl_for_each_suff_cost_rdp[sufficient_costs.index(sufficient_cost)]

xs_rdp = [i*(target_time/trotter_steps) for i in range(0, trotter_steps+1)] 

xlabel = 'Time'
ylabel = 'Probability of 110'

plt.plot(ts, probs_110, '-', label='Exact', clip_on=False)
plt.plot(xs_rdp, ys_rdp, 'o', label='Direct Trotter Implementation', clip_on=False)

controls = iplt.plot(xs_rdp, f_y, 'x', sufficient_cost=sufficient_costs, clip_on=False, label='Direct Trotter Implementation + ISL', slider_formats={"sufficient_cost": "{:.1e}"})

plt.legend(loc='best', prop={'size': 10})
plt.xlim(0,np.pi)
plt.xlabel(xlabel)
_ = plt.ylabel(ylabel)


Now device noise has been introduced, the standard Trotterisation procedure quickly fails. Looking across, as time increases, alongside the circuit depth, the observed probability decays exponentially such that by $t=1.0$ the qubits are being measured in a purely random state. Interestingly, this demonstrates the exponential relationship between noise and number of gates as well as the exponential decay of the qubits. Notably, the standard Trotterised circuits fails to capture any of the important physics of the Heisenberg model.

By contrast, the significantly shallower circuits produced by the ISL algorithm generate far less noise. The result of this is a simulation where the measured probability of the |110> is much closer to the form of the analytic solution. Importantly, the number of gates of our recompiled solution does not grow linearly with the simulation time and thus avoids the exponential decay of the observed signal.

Another result we can see here is the accuracy of the final solution with respect to the ISL sufficient cost, which is  (of the state produced by the ISL solution and original circuit respectively). At the furthest left of the slider, with sufficient cost 0.1, the ISL result is inaccurate, deviating at many points from the true solution. In this case, the approximately equivalent circuits produce states with only 90% overlap, and this large difference leads to different measured probabilities of the |110> state. Increasing the slider we see performance improve, with the stricter recompilation threshold leading to ISL solutions that more faithfully reproduce the dynamics. The accuracy of our solution appears to peak for a sufficient cost of 0.001, before plateauing and even reducing for higher cost. This effect is due to the aforementioned trade-off between overlap and number of gates required to obtain that accuracy. Now that gate noise is present, an increased number of gates has visible consequences for the accuracy of the final solution. Adjusting the slider, we see that the optimal sufficient cost for ISL is 0.001, which we use going forward. 

Despite this exciting indication on the fake device, the ISL solution is still far from perfect and deviates by up to 10% from the analytic solution. To improve this further we apply our next step of the solution: measurement calibration.

### 3-4 Measurement Calibration

Here we implement measurement calibration as described in the Qiskit tutorial (https://qiskit.org/documentation/tutorials/noise/3_measurement_error_mitigation.html). This is an extremely useful and well proven technique to factor out errors in the measurement of the qubits. Most importantly, this method of post-processing does not add more gates to the benchmark circuit which we will be executing later.

In [ ]:
backend_name = 'ibmq_jakarta'

target_time = np.pi
trotter_steps = min_trotter_steps_visual_convergence
num_qubits = 7
shots = 8192
reps = 8
sufficient_cost = 1e-3

f_isl_mc = file_utils.get_full_trotter_circuit_results_isl_meas_cal_filename('0','pi', trotter_steps, backend_name, shots, reps, sufficient_cost)

print('\nLoading files containing recompiled trotter circuit results with a trotter_step of {} and sufficient cost of {}'.format(trotter_steps, sufficient_cost))
print('    ' + f_isl_mc) 

# try and load existing results for recompiled trotter circuits and measurement calibration circuits
try:
    print('Existing results found using {}'.format(backend_name))
    results_for_each_rep_mc = np.load(f_isl_mc + '.npy', allow_pickle=True).tolist()
except FileNotFoundError:
    print('file not found {}'.format(f_isl_mc))
    

In [ ]:
prob_target_state_for_each_trotter_step = np.zeros(trotter_steps + 1)
num_meas_cal_circuits = 8
target_state = '110'

# extract measurement calibration results
meas_cal_results = results_for_each_rep_mc[0].results[0 : num_meas_cal_circuits]
cmf = mitigation_utils.do_measurement_calibration(_, num_qubits, [1,3,5], cal_results=Result(backend_name, 'BACKEND VERSION???', 'QOBJ???', 'JOB_ID???', True, meas_cal_results))

for rep in range(reps):
    prob_target_state = []
    results = results_for_each_rep_mc[rep]
    
    # extract trotter circuit results
    sim_results_1 = results.results[num_meas_cal_circuits : int((trotter_steps + 1) / 2) + num_meas_cal_circuits]
    sim_results_2 = results.results[int((trotter_steps + 1) / 2) + 2*num_meas_cal_circuits : int((trotter_steps + 1) / 2) + 2*num_meas_cal_circuits + int((trotter_steps + 1)/ 2) + 1]
    
    # apply measurement calibration
    corrected_sim_results = cmf.filter.apply(Result(backend_name, 'BACKEND VERSION???', 'QOBJ???', 'JOB_ID???', True, sim_results_1 + sim_results_2)) 
    
    # convert results to state probabilities
    state_probs_at_each_trotter_step = analysis_utils.get_state_probs_at_each_trotter_step(corrected_sim_results.get_counts(), 1)
    prob_target_state_for_each_trotter_step = prob_target_state_for_each_trotter_step + np.array([state_probs[target_state] if target_state in state_probs else 0 for state_probs in state_probs_at_each_trotter_step])
    
    print('\nResult for rep {}'.format(rep))
    print('Backend: {}'.format(results.backend_name))
    print('Job ID: {}'.format(results.job_id))

# average over the number of reps
ave_prob_target_state_for_each_trotter_step = prob_target_state_for_each_trotter_step / reps

In [ ]:
%matplotlib ipympl

xss_mc = [ts] + [[i*(target_time/trotter_steps) for i in range(0, trotter_steps+1)] for _ in range(1)]
yss_mc = [probs_110, ave_prob_target_state_for_each_trotter_step]
labels_mc = ['Exact', 'Direct Trotter Implementation + ISL (MC)']
markers_mc = ['-', 'gx']
title_mc = 'Max Trotter steps: {}, suff cost: {}, shots: {}. reps: {}'.format(trotter_steps, sufficient_cost, shots, reps)

# plot multiple datasets on same graph
misc_utils.xys_plot(plt, xss_mc, yss_mc, 'Time', 'Probability of 110', labels_mc, markers_mc, title_mc, alphas=[1 for i in range(len(xss_mc))])

The above graph shows the result of simulating 35 Trotter steps across the entire evolution on the real ibmq_jakarta device. Remarkably, our solution combining both ISL approximate circuit recompilation and measurement calibration is able to reproduce the form of the analytic solution, calculating the probability of the $|110\rangle$ state within on average 2.4 percentage points of the true value. This constitutes our attempt at achieving the ambitious goal of reproducing the complete dynamics of the XXX Heisenberg spin model on the real device. Whilst some inaccuracies remain, our results clearly demonstrate the interesting physics of the model under study and strongly agree with the analytic solution, retaining its symmetry. Most importantly, by simulating the entire graph on real quantum hardware, we hope to have demonstrated that our solution is general to any amount of evolved time.

### 3-5 Final Benchmark
Let's move onto the benchmark for the challenge: the fidelity of our noisy final state with respect to the 110 state at the final time. An interesting property of this particular point of the evolution, also a consequence of the periodicity of the model, is that the probability of the |110> state is 1. This means that ultimately, any accurate circuit reproducing the evolution will prepare the qubits exactly into the |110> state.

This has important consequences for our approximate recompilation solution. When recompiling the Trotterised solution with low enough Trotter error, the problem for our ISL solver can be reformulated as:

"Given a circuit that prepares the |110> state, find a set of gates which maps this back to the |000> state."

This is perhaps clearer when we restate the ISL cost function $C = 1 - |\langle 0 | V^{\dagger} U | 0 \rangle |^2$. Let's see the result of our recompilation.

In [ ]:
target_qubit_indices = [1,3,5]
target_time = np.pi
trotter_steps = min_trotter_steps_visual_convergence
num_qubits = 7

sufficient_cost = 1e-3
reps = 8
shots = 8192

quantum_register = QuantumRegister(num_qubits)
qc = QuantumCircuit(quantum_register)
# state prep of |110> state (little endian)
qc.x([3,5])

time_slice = target_time / trotter_steps if trotter_steps > 0 else 0

# build the trotter circuit 
qc = circuit_utils.build_trotter_circuit(qc, quantum_register, target_qubit_indices, circuit_utils.Trot_gate, trotter_steps, time_slice)
print('Original circuit')
print(qc)

print('\n Recompiled circuit')
recompiled_qc = circuit_utils.ISL_from_scratch(qc, sufficient_cost) # recompile using ISL
print(recompiled_qc['circuit'])

From above, we see the recompiled solution consists of a single X gate followed by a CNOT, effectively preparing the |110> state. Interestingly, this is not the lowest depth possible solution, since the same transformation could be made with just two X gates. However, as described previously, ISL incrementally builds an equivalent circuit with layers of thinly dressed CNOT gates. Whilst this allows the success seen earlier, producing shallow circuits able to reproduce any part of the dynamics, it means a solution with zero CNOT gates cannot be produced. In summary, this behaviour follows our desire for our solution to be general beyond the benchmark.

With the recompilation complete, we run the benchmark including measurement calibration.

In [ ]:
backend_name = 'ibmq_jakarta'
print('Running on: {}'.format(backend_name))
sufficient_cost = 1e-3
reps = 8
shots = 8192

f_bmark_isl = file_utils.get_partial_trotter_stomo_circuits_results_circuits_isl_filename('0','pi', trotter_steps, trotter_steps, backend_name, shots, reps, sufficient_cost) 

print('\nLoading files containing recompiled trotter circuit results with a trotter_step of {} and sufficient cost of {}'.format(trotter_steps, sufficient_cost))
print('    ' + f_bmark_isl) 
# load results and state tomography circuits for final benchmark
try:
    print('Existing results object found')
    results_circuits_bmark_isl = np.load(f_bmark_isl + '.npy', allow_pickle=True)
except FileNotFoundError:
    print('file not found {}'.format(f_bmark_isl)) 
    
results_bmark_isl = results_circuits_bmark_isl[0]
st_qcs = results_circuits_bmark_isl[1]

for i, result in enumerate(results_bmark_isl):
    print('\nResult for rep {}'.format(i+1))
    print('Backend: {}'.format(result.backend_name))
    print('Job ID: {}'.format(result.job_id))

f_cmf = file_utils.get_meas_cal_dir() + 'meas_fitter_final_stomo_reps_8_shots_8192_sc_0.001_62235daeb9d2d8f6e45c1e9b'
# load measurement calibration fitter 
# job_id: 62235daeb9d2d8f6e45c1e9b
cmf = np.load(f_cmf + '.npy', allow_pickle=True).tolist()
print(cmf)
# apply measurement calibration
corrected_results_bmark_isl = [cmf.filter.apply(result) for result in results_bmark_isl]

# compute fidelities with target state
fidelities_bmark_isl = [analysis_utils.state_tomo(corrected_result, st_qcs) for corrected_result in corrected_results_bmark_isl] 

print('***')
print('Time {}, backend {}, Trotter steps {}, shots {}, reps {}, ISL suff cost {}'.format(target_time, backend_name, trotter_steps, shots, reps, sufficient_cost))
print('Fidelity {} \u00B1 {:.4f}'.format(np.mean(fidelities_bmark_isl), np.std(fidelities_bmark_isl)))


Overall we are able to achieve a benchmark fidelity of 99.3%, through our combination of approximation circuit recompilation and measurement calibration. To see how each step of our solution has improved our results, we include additional tomography experiments and record the fidelity achieved with and without each of these techniques. The code used to obtain these additional values can be found in ibmq-qsim-challenge-comparisons.ipynb.

Table comparing results for 35 Trotter Steps




|             | Without ISL    | With ISL |
| ----------- | ----------- | ----------- |
| Without MC      | 0.119*   | 0.886       |
| With MC   | 0.121*        | 0.993        |



\* Indicates obtained on Fake Jakarta.

# 4 Run it yourself!

The 2 cells below allow you to run the code to reproduce our 2 primary results: 
    
1. The full dynamics of the Heisenberg XXX model starting in the $|110>$ state on a real quantum device 
    
2. The value obtained for the final fidelity benchmark. 

Nothing in these code blocks is loaded from file, the circuit recompilation and execution is done from scratch.

Connect to the IBM provider and connect to the real backend below. Uncomment the first line and add your api key if the notebook doesn't load your account automatically.

In [ ]:
#IBMQ.save_account(api_key, overwrite=True)  # replace TOKEN with your API token string (https://quantum-computing.ibm.com/lab/docs/iql/manage/account/ibmq)
provider = IBMQ.load_account()

# Get backend for experiment

provider = IBMQ.get_provider(hub='ibm-q-community', group='ibmquantumawards', project='open-science-22')

# backend = provider.get_backend('ibmq_jakarta') # Uncomment to run on real device
backend = AerSimulator.from_backend(FakeJakarta())

IBMQ.stored_account() # check the account 

In [ ]:
# set up the experiment variables
target_time = np.pi
trotter_steps = min_trotter_steps_visual_convergence
num_qubits = 7

print(backend.name())
sufficient_cost = 1e-3
reps = 8
shots = 8192

target_qubit_indices = [1,3,5]

# Reproduce the benchmark

In [ ]:
import logging

logging.getLogger("isl").setLevel(logging.DEBUG)

quantum_register = QuantumRegister(num_qubits)
qc = QuantumCircuit(quantum_register)
# state prep of |110> state (little endian)
qc.x([3,5])

# define time slice for trotterised time evolution
time_slice = target_time / trotter_steps if trotter_steps > 0 else 0 

# build trotter circuit
qc = circuit_utils.build_trotter_circuit(qc, quantum_register, target_qubit_indices, circuit_utils.Trot_gate, trotter_steps, time_slice)

# our ISL recompiler doesn't accept multi-qubit Pauli strings so transpile first
qc = transpile(qc, basis_gates=['u1','u2','u3','cx'], optimization_level=0)

# recompile using ISL and extract recompiled circuit from ISL results object
config = ISLConfig(sufficient_cost=sufficient_cost)
results = ISLRecompiler(qc, isl_config=config, backend=statevector).recompile()
recompiled_qc = results['circuit']

# construct the 27 circuits required for state tomography
st_qcs = state_tomography_circuits(recompiled_qc, target_qubit_indices)

# execute measurement calibration circuits
cmf = mitigation_utils.do_measurement_calibration(backend, num_qubits, [1,3,5])

# execute state tomography circuits
jobs_bmark_isl = job_utils.build_job_list(st_qcs, backend, shots, reps)

job_utils.monitor(jobs_bmark_isl)

results_bmark_isl = [job.result() for job in jobs_bmark_isl]

for i, result in enumerate(results_bmark_isl):
    print('Result for rep {}'.format(i+1))
    print('Backend: {}'.format(result.backend_name))
    print('Job ID: {}'.format(result.job_id))

# apply measurement calibration
corrected_results_bmark_isl = [cmf.filter.apply(result) for result in results_bmark_isl]
# compute fidelities with target state
fidelities_bmark_isl = [analysis_utils.state_tomo(corrected_result, st_qcs) for corrected_result in corrected_results_bmark_isl] 

print('***')
print('Time {}, backend {}, Trotter steps {}, shots {}, reps {}, ISL suff cost {}'.format(target_time, backend.name(), trotter_steps, shots, reps, sufficient_cost))
print('Fidelity {} \u00B1 {:.4f}'.format(np.mean(fidelities_bmark_isl), np.std(fidelities_bmark_isl)))


# Reproduce entire evolution

In [ ]:
%matplotlib ipympl

circuits_for_each_trotter_step = []
num_cnots = 0

for trotter_step in range(0, trotter_steps + 1): 
    print('{} out of {}.'.format(trotter_step, trotter_steps))
    quantum_register = QuantumRegister(num_qubits)
    qc = QuantumCircuit(quantum_register)
    # state prep of |110> state (little endian)
    qc.x([3,5])

    # define time slice for trotterised time evolution
    time_slice = target_time / trotter_steps if trotter_steps > 0 else 0
    
    # build trotter circuit
    qc = circuit_utils.build_trotter_circuit(qc, quantum_register, target_qubit_indices, circuit_utils.Trot_gate, trotter_step, time_slice)
    
    # our ISL recompiler doesn't accept multi-qubit Pauli strings so transpile first
    qc = transpile(qc, basis_gates=['u1','u2','u3','cx'], optimization_level=0)

    # recompile using ISL and extract recompiled circuit from ISL results object
    config = ISLConfig(sufficient_cost=sufficient_cost)
    results = ISLRecompiler(qc, isl_config=config, backend=statevector).recompile()
    recompiled_qc = results['circuit']
    
    recompiled_qc.add_register(ClassicalRegister(len(target_qubit_indices)))
    recompiled_qc.measure(target_qubit_indices, list(range(len(target_qubit_indices))))
    num_cnots += recompiled_qc.count_ops()['cx']
    [circuits_for_each_trotter_step.append(recompiled_qc) for _ in range(reps)]

print('ave cnots: {}'.format(num_cnots / (trotter_steps + 1)))
# execute circuits
batch_job_for_each_trotter_step = job_utils.build_batch_job_list(circuits_for_each_trotter_step, backend, shots)
job_utils.job_monitor(batch_job_for_each_trotter_step)

###
results_for_all_trotter_steps = batch_job_for_each_trotter_step.result()
# apply measurement calibration
corrected_results = cmf.filter.apply(results_for_all_trotter_steps)
# extract state probabilities from results
state_probs_at_each_trotter_step = analysis_utils.get_state_probs_at_each_trotter_step(corrected_results.get_counts(), reps)

target_state = '110'
# extract state probabilities for target state from complete set of state probabilities
prob_target_state_for_each_trotter_step = np.array([state_probs[target_state] if target_state in state_probs else 0 for state_probs in state_probs_at_each_trotter_step])
###
xss_mc = [ts] + [[i*(target_time/trotter_steps) for i in range(0, trotter_steps+1)]]
yss_mc = [probs_110, prob_target_state_for_each_trotter_step]
labels_mc = ['Exact', 'Direct Trotter Implementation + ISL (MC)']
markers_mc = ['-', 'gx']
title_mc = 'Max Trotter steps: {}, suff cost: {}, shots: {}. reps: {}'.format(trotter_steps, sufficient_cost, shots, reps)

# plot multiple datasets on same graph
misc_utils.xys_plot(plt, xss_mc, yss_mc, 'Time', 'Probability of 110', labels_mc, markers_mc, title_mc, alphas=[1 for i in range(len(xss_mc))])
    